In [37]:
from dotenv import load_dotenv
import os, requests, sys, json
from IPython.display import JSON
from pathlib import Path
import logging.handlers
from tqdm import tqdm
import pandas as pd
import numpy as np
from scipy.signal import convolve2d
import warnings
warnings.filterwarnings("ignore")

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from Helpers.Exception_Handling import Exception_Handling as eh
from Helpers.Mongo_Interface import Mongo_Interface as mi

env_path = Path(fr"{os.environ['USERPROFILE']}\.env") 
load_dotenv(dotenv_path=env_path)

True

In [108]:
results = mi.get_all()
len(results)

7323

In [109]:
horizontal_kernel = np.array([[ 1, 1, 1, 1]])
vertical_kernel = np.transpose(horizontal_kernel)
diag1_kernel = np.eye(4, dtype=np.uint8)
diag2_kernel = np.fliplr(diag1_kernel)
detection_kernels = [horizontal_kernel, vertical_kernel, diag1_kernel, diag2_kernel]

def winning_move(board, player):
    for kernel in detection_kernels:
        if (convolve2d(board == player, kernel, mode="valid") == 4).any():
            return True
    return False

In [110]:
columns = [item for item in range(1, 43)]
columns.append('board_series')
columns.append('winner')
columns.append('looser')
columns.append('is_stale')


df = pd.DataFrame(columns=columns)
obj =''
for i in tqdm(range(len(results))):
    flat_board = results[i]['steps'][-1:][0][0]['observation']['board'].copy()
    if len(results[i]['steps']) < 3:
           continue
    obj = first_player_move
    board = np.resize(flat_board, (6,7))
    one_winning_move = winning_move(board, 1)
    flat_board.append(results[i]['steps'])
    if winning_move(board, 1):
        flat_board.append(1)
        flat_board.append(2)
        flat_board.append(False)
    elif winning_move(board, 2):
        flat_board.append(2)
        flat_board.append(1)
        flat_board.append(False)
    else:
        flat_board.append(None)
        flat_board.append(None)
        flat_board.append(True)
    df2 = pd.DataFrame(data=[flat_board], columns=columns)
    df = pd.concat([df, df2], ignore_index=True)
df

100%|█████████████████████████████████████████████████████████████████████████████| 7323/7323 [00:10<00:00, 686.32it/s]


,1,2,3,4,5,6,7,8,9,10,...,37,38,39,40,41,42,board_series,winner,looser,is_stale
0,0,0,0,0,0,0,0,0,0,0,...,1,2,2,1,0,0,"[[{'action': 0, 'info': {}, 'observation': {'b...",1,2,False
1,0,0,1,0,0,0,0,0,0,2,...,0,2,1,1,2,0,"[[{'action': 0, 'info': {}, 'observation': {'b...",1,2,False
2,0,0,0,0,0,0,0,0,1,1,...,1,2,1,1,2,0,"[[{'action': 0, 'info': {}, 'observation': {'b...",1,2,False
3,0,0,2,0,0,0,0,0,2,1,...,2,1,1,1,2,2,"[[{'action': 0, 'info': {}, 'observation': {'b...",1,2,False
4,0,0,0,0,1,0,0,0,0,0,...,2,1,2,2,0,0,"[[{'action': 0, 'info': {}, 'observation': {'b...",1,2,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7299,0,0,0,0,0,0,0,0,0,0,...,1,1,1,2,0,2,"[[{'action': 0, 'info': {}, 'observation': {'b...",1,2,False
7300,2,2,1,1,0,2,2,1,1,2,...,2,1,1,1,2,2,"[[{'action': 0, 'info': {}, 'observation': {'b...",2,1,False
7301,2,2,0,2,2,0,2,1,1,0,...,2,2,1,1,1,2,"[[{'action': 0, 'info': {}, 'observation': {'b...",2,1,False
7302,1,0,1,1,2,2,0,2,0,2,...,0,2,1,1,1,2,"[[{'action': 0, 'info': {}, 'observation': {'b...",2,1,False


In [111]:
df[df['winner'] == 1]

,1,2,3,4,5,6,7,8,9,10,...,37,38,39,40,41,42,board_series,winner,looser,is_stale
0,0,0,0,0,0,0,0,0,0,0,...,1,2,2,1,0,0,"[[{'action': 0, 'info': {}, 'observation': {'b...",1,2,False
1,0,0,1,0,0,0,0,0,0,2,...,0,2,1,1,2,0,"[[{'action': 0, 'info': {}, 'observation': {'b...",1,2,False
2,0,0,0,0,0,0,0,0,1,1,...,1,2,1,1,2,0,"[[{'action': 0, 'info': {}, 'observation': {'b...",1,2,False
3,0,0,2,0,0,0,0,0,2,1,...,2,1,1,1,2,2,"[[{'action': 0, 'info': {}, 'observation': {'b...",1,2,False
4,0,0,0,0,1,0,0,0,0,0,...,2,1,2,2,0,0,"[[{'action': 0, 'info': {}, 'observation': {'b...",1,2,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7292,0,1,1,2,2,2,1,0,2,1,...,1,1,1,2,1,1,"[[{'action': 0, 'info': {}, 'observation': {'b...",1,2,False
7294,0,0,1,1,1,1,0,0,0,2,...,2,2,1,1,1,2,"[[{'action': 0, 'info': {}, 'observation': {'b...",1,2,False
7295,1,0,2,1,0,2,2,1,0,1,...,2,2,1,2,2,1,"[[{'action': 0, 'info': {}, 'observation': {'b...",1,2,False
7299,0,0,0,0,0,0,0,0,0,0,...,1,1,1,2,0,2,"[[{'action': 0, 'info': {}, 'observation': {'b...",1,2,False


In [127]:
columns2 = [item for item in range(1, 43)]
columns2.append('target')

df_actions = pd.DataFrame(columns=columns2)
df_winner_one = df[df['winner'] == 1]

preload = []

for i in tqdm(df_winner_one.index):
    series = df_winner_one.loc[i, 'board_series'].copy()
    if len(series) > 3:
        for j in range(2, len(series), 2):
            if len(series) > j + 1:
                flat_board = df_winner_one.loc[i, 'board_series'][j][0]['observation']['board'].copy()
                #print(len(flat_board))
                seen = np.resize(df_winner_one.loc[i, 'board_series'][j][0]['observation']['board'], (6,7))
                action = np.resize(df_winner_one.loc[i, 'board_series'][j + 1][0]['observation']['board'], (6,7))
                result = action - seen
                target = np.unravel_index(result.argmax(), result.shape)[1]
                flat_board.append(target)
                #print(flat_board)
                preload.append(flat_board)
                #df2 = pd.DataFrame(data=[flat_board], columns=columns2)

df_actions = pd.DataFrame(data=preload, columns=columns2)

df_actions

100%|████████████████████████████████████████████████████████████████████████████| 4148/4148 [00:01<00:00, 2103.42it/s]


,1,2,3,4,5,6,7,8,9,10,...,34,35,36,37,38,39,40,41,42,target
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,2,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,2,0,0,0,4
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,2,1,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,2,1,0,0,3
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,2,1,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46349,0,0,0,0,0,0,0,0,0,0,...,0,0,2,1,1,1,2,0,2,1
46350,0,0,0,0,0,0,0,0,0,0,...,0,0,2,1,1,1,2,0,2,1
46351,0,0,0,0,0,0,0,0,0,0,...,0,0,2,0,0,0,1,0,0,6
46352,0,0,0,0,0,0,0,0,0,0,...,0,0,2,0,2,0,1,0,1,5


In [129]:
df_actions['tick'] = 1

In [131]:
df_actions.groupby([item for item in range(1, 43)])['tick'].sum().sort_values(ascending=False)

1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42
0  0  0  0  0  0  0  0  0  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   2   0   0   0   0   0   0   1   0   0   0     707
                                                           2   0   0   0   0   0   0   1   0   0   0   0   0   0   2   0   0   0   0   0   0   1   0   0   0     338
                                                           0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   1   0   0   2     241
                                                                                                                                           2   1   0   0   0     241
                                                                                                                                           0   1   2   0   0     201
                 

In [100]:




seen = np.resize(df.loc[0, 'board_series'][2][0]['observation']['board'], (6,7))
action = np.resize(df.loc[0, 'board_series'][3][0]['observation']['board'], (6,7))
result = action - seen
target = np.unravel_index(result.argmax(), result.shape)[1]
print(target)

1


In [9]:
JSON(obj)

<IPython.core.display.JSON object>